In [13]:
import pandas as pd
from IPython.display import display
from DMRBChange import DMRBChange
import numpy as np

#I1 = input('\n\nEnter file name (put in input folder) : ')
pathname = "LL3X data.xlsx" # "input/"+I1

# -------- READ EXCEL SPREADSHEET ----- #
xl = pd.ExcelFile(pathname)
Tabstr = xl.sheet_names  # see all sheet names
display(Tabstr[0])
Tabs = []
for count, i in enumerate(Tabstr):
    Tabs.append(xl.parse(Tabstr[count]))
# -------- READ EXCEL SPREADSHEET ----- #

Tab1 = pd.DataFrame(Tabs[0])
Tab1['ST_CH'], Tab1['LT_CH'] = Tab1['DSO']-Tab1['DMO'], Tab1['DSD']-Tab1['DMO']
Tab1['ST_CH_GC'], Tab1['LT_CH_GC'] = abs(Tab1['ST_CH']), abs(Tab1['LT_CH'])
Tab1 = np.round(Tab1,1)
Columns = list(Tab1.columns)

Tab1STLBC1 = Tab1[Tab1['ST_CH'] == Tab1.groupby('BLD')['ST_CH'].transform('max')]
Tab1STLBC = pd.DataFrame(Tab1STLBC1[Tab1STLBC1['DSO'] == Tab1STLBC1.groupby('BLD')['DSO'].transform('max')]).drop_duplicates(subset="BLD", keep = "first")
Tab1LTLBC1 = Tab1[Tab1['LT_CH'] == Tab1.groupby('BLD')['LT_CH'].transform('max')]
Tab1LTLBC = pd.DataFrame(Tab1LTLBC1[Tab1LTLBC1['DSD'] == Tab1LTLBC1.groupby('BLD')['DSD'].transform('max')]).drop_duplicates(subset="BLD", keep = "first")

Tab1STGC1 = Tab1[Tab1['ST_CH_GC'] == Tab1.groupby('BLD')['ST_CH_GC'].transform('max')]
Tab1STGC = pd.DataFrame(Tab1STGC1[Tab1STGC1['DSO'] == Tab1STGC1.groupby('BLD')['DSO'].transform('max')]).drop_duplicates(subset="BLD", keep = "first")
Tab1LTGC1 = Tab1[Tab1['LT_CH_GC'] == Tab1.groupby('BLD')['LT_CH_GC'].transform('max')]
Tab1LTGC = pd.DataFrame(Tab1LTGC1[Tab1LTGC1['DSD'] == Tab1LTGC1.groupby('BLD')['DSD'].transform('max')]).drop_duplicates(subset="BLD", keep = "first")

ST, LT = [5, 3, 1, 0, -1, -3, -5], [10, 5, 3, 0, -3, -5, -10]

Tab1STLBCRES, Tab1STLBCOSR = Tab1STLBC[Tab1STLBC['SNSTV']=='RES'], Tab1STLBC[Tab1STLBC['SNSTV']=='OSR']
Tab1LTLBCRES, Tab1LTLBCOSR = Tab1LTLBC[Tab1LTLBC['SNSTV']=='RES'], Tab1LTLBC[Tab1LTLBC['SNSTV']=='OSR']
Tab1STGCRES, Tab1STGCOSR = Tab1STGC[Tab1STGC['SNSTV']=='RES'], Tab1STGC[Tab1STGC['SNSTV']=='OSR']
Tab1LTGCRES, Tab1LTGCOSR = Tab1LTGC[Tab1LTGC['SNSTV']=='RES'], Tab1LTGC[Tab1LTGC['SNSTV']=='OSR']
    
DMRBChange(Tab1STLBCRES, 'ST_CH', ST)
DMRBChange(Tab1STLBCOSR, 'ST_CH', ST)
DMRBChange(Tab1LTLBCRES, 'LT_CH', LT)
DMRBChange(Tab1LTLBCOSR, 'LT_CH', LT)

DMRBChange(Tab1STGCRES, 'ST_CH', ST)
DMRBChange(Tab1STGCOSR, 'ST_CH', ST)
DMRBChange(Tab1LTGCRES, 'LT_CH', LT)
DMRBChange(Tab1LTGCOSR, 'LT_CH', LT)

STDayDwellLBC = pd.DataFrame(Tab1STLBCRES[Tab1STLBCRES.columns.difference(Columns)].sum(), columns=['ST Day RES LBC'])
STDayOSRLBC = pd.DataFrame(Tab1STLBCOSR[Tab1STLBCOSR.columns.difference(Columns)].sum(), columns=['ST Day OSR LBC'])
LTDayDwellLBC = pd.DataFrame(Tab1LTLBCRES[Tab1LTLBCRES.columns.difference(Columns)].sum(), columns=['LT Day RES LBC'])
LTDayOSRLBC = pd.DataFrame(Tab1LTLBCOSR[Tab1LTLBCOSR.columns.difference(Columns)].sum(), columns=['LT Day OSR LBC'])

STDayDwellGC = pd.DataFrame(Tab1STGCRES[Tab1STGCRES.columns.difference(Columns)].sum(), columns=['ST Day RES GC'])
STDayOSRGC = pd.DataFrame(Tab1STGCOSR[Tab1STGCOSR.columns.difference(Columns)].sum(), columns=['ST Day OSR GC'])
LTDayDwellGC = pd.DataFrame(Tab1LTGCRES[Tab1LTGCRES.columns.difference(Columns)].sum(), columns=['LT Day RES GC'])
LTDayOSRGC = pd.DataFrame(Tab1LTGCOSR[Tab1LTGCOSR.columns.difference(Columns)].sum(), columns=['LT Day OSR GC'])

DMRB_ST = STDayDwellLBC.join(STDayOSRLBC).join(STDayDwellGC).join(STDayOSRGC)
DMRB_LT = LTDayDwellLBC.join(LTDayOSRLBC).join(LTDayDwellGC).join(LTDayOSRGC)

DMRB_ST.loc["Total"] = DMRB_ST.sum()
DMRB_LT.loc["Total"] = DMRB_LT.sum()

display(DMRB_ST.reindex(['<=-5','>-5<=-3', '>-3<=-1', '>=-1<0', '=0','>0<1','>=1<3','>=3<5','>=5','Total']))
display(DMRB_LT.reindex(['<=-10','>-10<=-5', '>-5<=-3', '>=-3<0', '=0','>0<3','>=3<5','>=5<10','>=10','Total']))

'LL3X data'

,ST Day RES LBC,ST Day OSR LBC,ST Day RES GC,ST Day OSR GC
<=-5,0,0,28,0
>-5<=-3,21,0,216,0
>-3<=-1,1097,0,2307,0
>=-1<0,3098,0,3039,0
=0,455,0,43,0
>0<1,4646,0,3731,0
>=1<3,3696,0,3649,0
>=3<5,641,0,641,0
>=5,107,0,107,0
Total,13761,0,13761,0


,LT Day RES LBC,LT Day OSR LBC,LT Day RES GC,LT Day OSR GC
<=-10,0,0,0,0
>-10<=-5,0,0,23,0
>-5<=-3,0,0,42,0
>=-3<0,1702,0,2453,0
=0,231,0,29,0
>0<3,10363,0,9749,0
>=3<5,1324,0,1324,0
>=5<10,141,0,141,0
>=10,0,0,0,0
Total,13761,0,13761,0
